In [15]:
from pathlib import Path

BASEDIR = Path('.').resolve().parent.parent.parent
DIR = BASEDIR.joinpath('download_viirs_iridis/download_viirs_iridis/datain/2016')
rasters = [[x for x in y.iterdir() if x.name.endswith('rade9h.tif') if '75N180W' in x.name][0] for y in DIR.iterdir() if not y.name == 'ANNUAL_Composite_2016']
sorted(rasters)

[PosixPath('/jupyter_test/download_viirs_iridis/download_viirs_iridis/datain/2016/01/SVDNB_npp_20160101-20160131_75N180W_vcmcfg_v10_c201603132032.avg_rade9h.tif'),
 PosixPath('/jupyter_test/download_viirs_iridis/download_viirs_iridis/datain/2016/02/SVDNB_npp_20160201-20160229_75N180W_vcmcfg_v10_c201603152010.avg_rade9h.tif'),
 PosixPath('/jupyter_test/download_viirs_iridis/download_viirs_iridis/datain/2016/03/SVDNB_npp_20160301-20160331_75N180W_vcmcfg_v10_c201604191144.avg_rade9h.tif'),
 PosixPath('/jupyter_test/download_viirs_iridis/download_viirs_iridis/datain/2016/04/SVDNB_npp_20160401-20160430_75N180W_vcmcfg_v10_c201606140957.avg_rade9h.tif'),
 PosixPath('/jupyter_test/download_viirs_iridis/download_viirs_iridis/datain/2016/05/SVDNB_npp_20160501-20160531_75N180W_vcmcfg_v10_c201606281430.avg_rade9h.tif'),
 PosixPath('/jupyter_test/download_viirs_iridis/download_viirs_iridis/datain/2016/06/SVDNB_npp_20160601-20160630_75N180W_vcmcfg_v10_c201608101832.avg_rade9h.tif'),
 PosixPath('/jup

In [16]:
import rasterio
import fiona
import rasterio.mask
import numpy as np
from pathlib import Path
BASEDIR = Path('.').resolve().parent.parent.parent
shp = BASEDIR.joinpath('viirs_preprocessing_analysis/viirs_preprocessing_analysis/shps/L0/HTI/hti_level0_2000_2020.shp')
raster = BASEDIR.joinpath('download_viirs_iridis/download_viirs_iridis/datain/2012/04/SVDNB_npp_20120401-20120430_75N180W_vcmcfg_v10_c201605121456.avg_rade9h.tif')

with fiona.open(shp, 'r') as s:
    features = [feature["geometry"] for feature in s]

with rasterio.open(raster) as r:
    out_image, out_transform = rasterio.mask.mask(r, features, nodata=-99999, crop=True)
    out_meta = r.meta.copy()
out_meta.update({
    "driver": "GTiff",
    "height": out_image.shape[1],
    "width": out_image.shape[2],
    "transform": out_transform,
    "count": 1,
    "compress": 'lzw',
    "predictor": 2,
    "bigtiff": "yes",
    'nodata': -99999
})

with rasterio.open('TEST_nd.tif', 'w', **out_meta) as dst:
    dst.write(out_image)